In [1]:
!which python; python -V;
from astropy import units as u
import numpy as np

from poliastro.bodies import Earth, Mars, Sun
from poliastro.twobody import Orbit

# Needed to handle TLE into Poliastro's Orbit
from tletools import TLE

# Needed for defining manouvers
from poliastro.maneuver import Maneuver
from poliastro.twobody import thrust
from poliastro.twobody.propagation import cowell
from poliastro.plotting import OrbitPlotter2D, OrbitPlotter3D

from perylune.orbit_tools import *

/home/thomson/devel/perylune/venv/bin/python
Python 3.8.5


In [2]:
# STEP 0: DEFINE TARGET ORBIT
# Let's get the TLE orbital data for the target satellite. As an example, let's pick a dead NOAA-17 sat.
# Its TLE data can be obtained from many places, such as celestrak or n2yo (https://www.n2yo.com/satellite/?s=27453)
tle_text = """NOAA-17
1 27453U 02032A   20263.80942421 -.00000011 +00000-0 +13583-4 0  9998
2 27453 098.5909 208.3215 0011096 327.5463 032.5033 14.25072668948324"""
tle_lines = tle_text.strip().splitlines()
tle_target = TLE.from_lines(*tle_lines)
orb_target = tle_target.to_orbit()

In [3]:
# STEP 1: LAUNCH
# Let's use ANDESITE sat as an example. It was launched as shareride by RocketLab's Electron on
# the latest rideshare launch on 2020 June 13. NORAD ID is 45726. 
# TLE elements taken from n2yo website, details https://www.n2yo.com/satellite/?s=45726

tle_text = """GDASAT-1
1 45726U 20037D   20278.45278018  .00000608  00000-0  65390-4 0  9991
2 45726  97.7132  96.2906 0012962 283.6573  76.3213 14.92011802 14275"""
tle_lines = tle_text.strip().splitlines()
tle1 = TLE.from_lines(*tle_lines)
orb1 = tle1.to_orbit()

In [4]:
# Let's get the details about initial orbit
print_orb(orb1)

6961 x 6979 km x 97.7 deg (GCRS) orbit around Earth (♁) at epoch 2020-10-04T10:52:00.207552000 (UTC)
a(𝑎)=6969.80km, b=6969.79km, e=0.00, i=97.71deg raan(Ω)=96.29deg argp(𝜔)=283.66deg nu(𝜈)=76.47deg
period=5790.84s perapis=6961km(582.63km) apoapsis=6979km(600.70km)


In [5]:
# Let's take a look at the target orbit
print_orb(orb_target)

7178 x 7194 km x 98.6 deg (GCRS) orbit around Earth (♁) at epoch 2020-09-19T19:25:34.251744000 (UTC)
a(𝑎)=7186.39km, b=7186.38km, e=0.00, i=98.59deg raan(Ω)=208.32deg argp(𝜔)=327.55deg nu(𝜈)=32.57deg
period=6062.85s perapis=7178km(800.27km) apoapsis=7194km(816.22km)


In [6]:
# Let's visualize the orbits

# Obtain the orbit after applying Hohmann transfer.
plot = OrbitPlotter3D()
plot.plot(orb1, label="GDASAT-1, after orbital insertion")
plot.plot(orb_target, label="NOAA-17, target sat")
plot.show()

In [7]:
# STEP 2: Synchronize Right Ascension of the Ascending Node. Changing RAAN as any other out of plane maneuver
# is in general a costly operation. However, there's a great saving trick that allows performing
# such change almost for free - the J2 perturbation (oblate earth)

# First, lets estimate how fast the orbit precesses in the course of 24 hours.

from poliastro.core.perturbations import J2_perturbation
tof = (24.0 * u.h).to(u.s)
orb24h = orb1.propagate(tof, method=cowell, ad=J2_perturbation, J2=Earth.J2.value, R=Earth.R.to(u.km).value)
print("Orbit precessed from RAAN=%s to %s in 24h, giving the rate of %s/24h." % (orb1.raan, orb24h.raan, (orb24h.raan - orb1.raan).to(u.deg)))

# The target orbit has RAAN of 208.32deg. Our initial orbit has RANN of 96.29deg. The orbital plane needs to rotate by 112.03deg.
# Since the perfromance is 0.988 deg/24h, the rotation will take roughly 113 days. Let's do the 113 days in one simulation...
tof = (113 * 24 * u.h).to(u.s)
orb2 = orb1.propagate(tof, method=cowell, ad=J2_perturbation, J2=Earth.J2.value, R=Earth.R.to(u.km).value)

# ... and then continue simulating one hour at a time until the RAAN is at least the same as that of target's orbit:
while orb2.raan < orb_target.raan:
    tof = (1*u.h).to(u.s)
    orb2 = orb2.propagate(tof, method=cowell, ad=J2_perturbation, J2=Earth.J2.value, R=Earth.R.to(u.km).value)

print("Orbit after J2 perturbation:")
print_orb(orb2)

print("Target orbit:")
print_orb(orb_target)

Orbit precessed from RAAN=96.2906 deg to 1.6978461805208191 rad in 24h, giving the rate of 0.9888204062498249 deg/24h.
Orbit after J2 perturbation:
6962 x 6975 km x 97.7 deg (GCRS) orbit around Earth (♁) at epoch 2021-01-26T05:52:00.207552000 (UTC)
a(𝑎)=6968.51km, b=6968.51km, e=0.00, i=97.71deg raan(Ω)=208.35deg argp(𝜔)=248.09deg nu(𝜈)=127.11deg
period=5789.23s perapis=6962km(584.12km) apoapsis=6975km(596.63km)
Target orbit:
7178 x 7194 km x 98.6 deg (GCRS) orbit around Earth (♁) at epoch 2020-09-19T19:25:34.251744000 (UTC)
a(𝑎)=7186.39km, b=7186.38km, e=0.00, i=98.59deg raan(Ω)=208.32deg argp(𝜔)=327.55deg nu(𝜈)=32.57deg
period=6062.85s perapis=7178km(800.27km) apoapsis=7194km(816.22km)


In [8]:
# Let's visualize the orbits. The virtually identical RAAN value makes the orbits almost co-planar. However,
# the only difference is their slightly different inclination: 98.59deg vs 97.71 deg.
plot = OrbitPlotter3D()
plot.plot(orb2, label="GDASAT-1, after RAAN drift")
plot.plot(orb_target, label="NOAA-17, target sat")
plot.show()

In [9]:
# STEP 3: Change orbital inclination.
# Our orbit has inclination of 97.7 deg, while the target is 98.6 deg. Let's calculate the delta inclination
delta_theta = orb_target.inc - orb1.inc

# Now let's wait till the spacecraft flies to the descending node.
orb3 = propagate_to_desc_node(orb2)

man1 = plane_change_maneuver(orb2, delta_theta)
orb4 = orb3.apply_maneuver(man1)

print("== Current orbit (after inclination change) ==")
print_orb(orb4)

print("== Target orbit ==")
print_orb(orb_target)

# The orbits are now fully co-planar!
# We can use Hohmann transfer now!

== Current orbit (after inclination change) ==
6964 x 6980 km x 98.6 deg (GCRS) orbit around Earth (♁) at epoch 2021-01-26T06:36:13.207443270 (UTC)
a(𝑎)=6971.72km, b=6971.72km, e=0.00, i=98.59deg raan(Ω)=208.35deg argp(𝜔)=227.35deg nu(𝜈)=-47.35deg
period=5793.23s perapis=6964km(585.40km) apoapsis=6980km(601.77km)
== Target orbit ==
7178 x 7194 km x 98.6 deg (GCRS) orbit around Earth (♁) at epoch 2020-09-19T19:25:34.251744000 (UTC)
a(𝑎)=7186.39km, b=7186.38km, e=0.00, i=98.59deg raan(Ω)=208.32deg argp(𝜔)=327.55deg nu(𝜈)=32.57deg
period=6062.85s perapis=7178km(800.27km) apoapsis=7194km(816.22km)


In [10]:
plot = OrbitPlotter2D()
plot.plot(orb4, label="GDASAT-1, after inclination change")
plot.plot(orb_target, label="NOAA-17, target sat")
plot.show()